<a href="https://colab.research.google.com/github/rajy4683/EVAP2/blob/master/ENDS9_TF_AmbigQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Jan  7 17:38:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np
import pandas as pd 
import json
import random
import math
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Then set a random seed for deterministic results/reproducability.

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!pip install jsonlines
import jsonlines

In [ ]:
with jsonlines.open('/content/dev_rand_split.jsonl') as reader:
    # reader.read()
    for out in reader:
        print(out)

In [ ]:
fout = open(f"openqatrain.tsv", "w+")
fout_meta = open(f"openqatrain_meta.tsv", "w+")
with open("/content/train.jsonl") as f:
    for line in f.readlines():
        json_line = json.loads(line)
        question = json_line['question']['stem']
        choices = json_line['question']['choices']
        candidates = " ".join([f"({c['label']}) {c['text']}" for c in choices]).replace("\n", " ")
        print(json_line)
        answer_key = json_line['answerKey']
        answer_key_idx = ord(answer_key[0]) - ord('A')
        answer_text = choices[answer_key_idx]['text']
        id = "OBQA_" + json_line['id']
        para = ""
        # if with_para:
        #     para = "\\n" + oyvind_paragraphs[id].replace("\n", " ").replace("\t", " ")
        fout.write(f"{question} \\n {candidates}{para}\t{answer_text}\n")
        fout_meta.write(f"{json_line['id']}\t{answer_key[0]}\n")

In [ ]:
def read_file(file, split):
    fout = open(f"./{split}.tsv", "w")
    fout_meta = open(f"./{split}_meta.tsv", "w")
    with open(file) as f:
        for line in f.readlines():
            json_line = json.loads(line)

            candidates_str = " ".join([f"({x['label']}) {x['text']}" for x in json_line['question']['choices']])
            if split != "test":
                selected_ans_string = [x['text'] for x in json_line['question']['choices'] if
                                        json_line['answerKey'] == x['label']]
                assert len(selected_ans_string) == 1, f"{len(selected_ans_string)} -- {json_line['answerKey']}"
            json_line['question']['stem'] = json_line['question']['stem'].replace("\t", " ").replace("\n", "")
            candidates_str = candidates_str.replace("\t", " ").replace("\n", "")

            if split == "test":
                fout_meta.write(f"{json_line['id']}\t-\n")
                fout.write(f"{json_line['question']['stem']} \\n {candidates_str}\t-\n")
            else:
                fout_meta.write(f"{json_line['id']}\t{json_line['answerKey']}\n")
                selected_ans_string[0] = selected_ans_string[0].replace("\t", " ").replace("\n", "")
                fout.write(f"{json_line['question']['stem']} \\n {candidates_str}\t{selected_ans_string[0]}\n")

In [ ]:
read_file("/content/CommonQA/train_rand_split.jsonl", "dev")

In [ ]:
sentence = []
label = []
with open('/content/CommonQA/train_rand_split.jsonl') as h:
    for line in h:
        example = json.loads(line)
        scores = []
        merged_choices = ' A: '.join([choice['text'] for choice in example['question']['choices']])
        input = 'Q: ' + example['question']['stem'] + ' A: ' + merged_choices
        correct_answer = [ choice['text'] for choice in example['question']['choices'] if choice['label'] == example['answerKey'] ][0]
        sentence.append(input)
        label.append(correct_answer)
        #print(input, correct_answer)
dataset_df = pd.DataFrame({'sentence':sentence, 'label':label})

In [ ]:
sentence = []
label = []
with open('/content/OpenBookQa/train.jsonl') as h:
    for line in h:
        example = json.loads(line)
        print(example.keys())
        # scores = []
        merged_choices = ' A: '.join([choice['text'] for choice in example['question']['choices']])
        input = 'Q: ' + example['question']['stem'] + ' A: ' + merged_choices
        correct_answer = [ choice['text'] for choice in example['question']['choices'] if choice['label'] == example['answerKey'] ][0]
        sentence.append(input)
        label.append(correct_answer)
        #print(input, correct_answer)
dataset_df = pd.DataFrame({'sentence':sentence, 'label':label})

In [7]:
dataset_df

,sentence,label
0,Q: The sun is responsible for A: puppies learn...,"plants sprouting, blooming and wilting"
1,Q: When standing miles away from Mount Rushmor...,the mountains seem smaller than in photographs
2,Q: When food is reduced in the stomach A: the ...,nutrients are being deconstructed
3,Q: Stars are A: warm lights that float A: made...,great balls of gas burning billions of miles away
4,Q: You can make a telescope with a A: straw A:...,mailing tube
...,...,...
3409,Q: Bears instinctively know when its time to h...,Their parents
3410,Q: An example of a fossil is a paw print in wh...,hard stones
3411,Q: Which is likeliest to make light pass throu...,any kind of tangible object
3412,Q: What most likely caused the castle built by...,very strong eddies of fast moving air carried ...


In [4]:
import itertools
import json
import os
import csv
import errno
import random
from random import shuffle
from typing import List
import spacy
from tqdm import tqdm

In [9]:
!unzip /content/ambignq_light.zip

Archive:  /content/ambignq_light.zip
   creating: ambignq_light/
  inflating: ambignq_light/dev_light.json  
  inflating: ambignq_light/LICENSE   
  inflating: ambignq_light/train_light.json  
  inflating: dev_light.json          
  inflating: LICENSE                 
  inflating: train_light.json        


In [5]:
def ambigqa():
    def read_file(file, dir, split):
        outfile = open(f"{dir}/{split}.tsv", "w")
        outfile_meta = open(f"{dir}/{split}_meta.tsv", "w")
        size = 0
        with open(file, "r") as f:
            json_file = json.load(f)
            for item in tqdm(json_file):
                question = item['question'].replace("\n", " ").replace("\t", " ")
                single_answers_already_included = []
                for anno in item["annotations"]:
                    if anno['type'] == "singleAnswer":
                        for ans in anno['answer']:
                            if ans not in single_answers_already_included:
                                ans = ans.replace("\n", " ").replace("\t", " ")
                                outfile.write(f"{question}\t{ans}\n")
                                outfile_meta.write(item['id'] + "\n")
                                single_answers_already_included.append(ans)
                                size += 1
                    else:
                        answers = []
                        for x in anno['qaPairs']:
                            answers.append(x['answer'][0])

                        answers = [x.strip() for x in answers]
                        answers = list(set(answers))  # to drop duplicate answers
                        for i, ordering in enumerate(itertools.permutations(answers)):
                            if i >= 3:
                                break
                            ans_str = " [SEP] ".join(ordering).replace("\n", " ").replace("\t", " ")
                            outfile.write(f"{question}\t{ans_str}\n")
                            outfile_meta.write(item['id'] + "\n")
                            size += 1
        return size

    count_dev = read_file("/content/ambignq_light/dev_light.json", "/content/ambigqa", "dev")
    count_train = read_file("/content/ambignq_light/train_light.json", "/content/ambigqa", "train")
    count_test = 0

    # Create TSVs and get counts.
    with open("ambigqa/counts.json", "w") as outfile:
        json.dump({"train": count_train, "dev": count_dev, "test": count_test}, outfile)

In [6]:
ambigqa()

100%|██████████| 10036/10036 [00:00<00:00, 219098.26it/s]


In [7]:
#dataset = pd.read_csv("/content/dev.tsv", sep='\t', header=None)
dataset = pd.read_csv("/content/ambigqa/train.tsv", sep='\t', header=None)
dataset.columns = ["sentence", "label"]

In [8]:
dataset

,sentence,label
0,When did the simpsons first air on television?,"April 19, 1987 [SEP] December 17, 1989"
1,When did the simpsons first air on television?,"December 17, 1989 [SEP] April 19, 1987"
2,Who played george washington in the john adams...,David Morse
3,What is the legal age of marriage in usa?,0 [SEP] 21 [SEP] 18 years of age [SEP] 19
4,What is the legal age of marriage in usa?,0 [SEP] 21 [SEP] 19 [SEP] 18 years of age
...,...,...
19801,Who was the last person in the uk to be executed?,Gwynne Evans and Peter Allen
19802,Who does wonder woman end up with in the comics?,Superman and Batman [SEP] General Steven Rockw...
19803,Who does wonder woman end up with in the comics?,General Steven Rockwell Trevor [SEP] Superman ...
19804,When were the first pair of jordans released?,"early 1984 [SEP] November 17, 1984"


In [ ]:
!gsutil cp -r \
  gs://unifiedqa/data/qasc/ \
  gs://unifiedqa/data/qasc_test/ \
  gs://unifiedqa/data/qasc_with_ir/ \
  gs://unifiedqa/data/qasc_with_ir_test/ \
  .

Copying gs://unifiedqa/data/qasc/counts.json...
Copying gs://unifiedqa/data/qasc/dev.tsv...
Copying gs://unifiedqa/data/qasc/dev_meta.tsv...
Copying gs://unifiedqa/data/qasc/test.tsv...
- [4 files][352.9 KiB/352.9 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://unifiedqa/data/qasc/test_meta.tsv...
Copying gs://unifiedqa/data/qasc/train.tsv...
Copying gs://unifiedqa/data/qasc/train_meta.tsv...
Copying gs://unifiedqa/data/qasc_test/counts.json...
Copying gs://unifiedqa/data/qasc_test/dev.tsv...
Copying gs://unifiedqa/data/qasc_test/dev_meta.tsv...
Copying gs://unifiedqa/data/qasc_test/test.tsv...
Copying gs://unifiedqa/data/qasc_test/test_meta.tsv...
Copying gs://unifiedqa/data/qasc_test/train.tsv...
Copying gs://unifiedqa

In [ ]:
!gsutil -m cp -r gs://unifiedqa/data/squad2/dev.tsv .

Copying gs://unifiedqa/data/squad2/dev.tsv...
/ [1/1 files][ 10.3 MiB/ 10.3 MiB] 100% Done                                    
Operation completed over 1 objects/10.3 MiB.                                     


In [ ]:
!gsutil -m cp -r \
  gs://unifiedqa/data/ambigqa/ \
  .

Copying gs://unifiedqa/data/ambigqa/counts.json...
Copying gs://unifiedqa/data/ambigqa/dev.tsv...
Copying gs://unifiedqa/data/ambigqa/train_meta.tsv...
Copying gs://unifiedqa/data/ambigqa/train.tsv...
Copying gs://unifiedqa/data/ambigqa/dev_meta.tsv...
/ [5/5 files][  2.8 MiB/  2.8 MiB] 100% Done                                    
Operation completed over 5 objects/2.8 MiB.                                      


In [ ]:
read_file("/content/dev_rand_split.jsonl", "dev")

Instantiate our German and English spaCy models.

In [9]:
!python3 -m spacy download de_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [10]:
import de_core_news_sm

In [11]:
spacy_de =  de_core_news_sm.load()
spacy_en = spacy.load('en')

Previously we reversed the source (German) sentence, however in the paper we are implementing they don't do this, so neither will we.

In [12]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Create our fields to process our data. This will append the "start of sentence" and "end of sentence" tokens as well as converting all words to lowercase.

In [ ]:
# SRC = Field(tokenize=tokenize_de, 
#             init_token='<sos>', 
#             eos_token='<eos>', 
#             lower=True)
SRC = Field(tokenize=tokenize_de, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

TRG = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [ ]:
from IPython.display import display, HTML
display(HTML(dataset.to_html()))

In [91]:
dataset[18880:18889]

,sentence,label
18880,What grade does high school start in japan?,seven [SEP] ten
18881,What grade does high school start in japan?,ten [SEP] seven
18882,Where is blanche from in streetcar named desire?,Laurel
18883,Where is blanche from in streetcar named desire?,"Laurel, Mississippi"
18884,Which two plates meet along the west coast of ...,Pacific Plate [SEP] North American Plate
18885,Which two plates meet along the west coast of ...,North American Plate [SEP] Pacific Plate
18886,What is the current version of php and mysql?,7.4.2 [SEP] 8.0.18
18887,What is the current version of php and mysql?,8.0.18 [SEP] 7.4.2
18888,When are you supposed to bring elf on the shel...,Thanksgiving to Christmas Eve


In [13]:
from torchtext import data 
import matplotlib.pyplot as plt
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =False, include_lengths=False, lower=False)
Label = data.Field(sequential =True, tokenize ='spacy', is_target=False, batch_first =False, include_lengths=False, lower=False)

In [14]:
#dataset = dataset_df
fields = [('sentence', Sentence),('label',Label)]
# example = [data.Example.fromlist([dataset.sentence[i],dataset.label[i]], fields) for i in range(dataset.shape[0])] 
example = [data.Example.fromlist([dataset.sentence[i],dataset.label[i]], fields) for i in range(10000)] 

commonqa_ds = data.Dataset(example, fields)
(train, valid) = commonqa_ds.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [15]:
Sentence.build_vocab(train)
Label.build_vocab(train)

In [ ]:
SRC.sequential, SRC.batch_first, SRC.include_lengths, SRC.is_target, TRG.sequential, TRG.batch_first, TRG.include_lengths, TRG.is_target

(True, False, False, False, True, False, False, False)

In [16]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  6784
Size of label vocab :  9945
Top 10 words appreared repeatedly : [('?', 8500), ('the', 7320), ('Who', 3537), ('in', 3129), ('of', 2834), ('is', 1729), ('When', 1711), ('did', 1218), ('What', 1038), ('was', 1031)]
Labels :  defaultdict(<function _default_unk_index at 0x7f25c1ca2ea0>, {'<unk>': 0, '<pad>': 1, '[': 2, ']': 3, 'SEP': 4, ',': 5, 'and': 6, 'the': 7, 'of': 8, '2017': 9, '-': 10, '"': 11, 'May': 12, 'The': 13, '2018': 14, 'October': 15, 'September': 16, 'March': 17, 'November': 18, 'July': 19, 'April': 20, 'December': 21, 'June': 22, '2016': 23, 'January': 24, 'August': 25, '1': 26, '2015': 27, 'John': 28, 'in': 29, 'New': 30, '6': 31, '10': 32, '2': 33, '17': 34, '12': 35, '15': 36, "'s": 37, 'February': 38, 'United': 39, '13': 40, '25': 41, '5': 42, 'to': 43, '8': 44, '2014': 45, 'James': 46, '26': 47, '7': 48, '18': 49, '4': 50, '9': 51, 'Michael': 52, 'a': 53, '20': 54, '11': 55, '3': 56, '16': 57, '14': 58, '21': 59, 'States': 60, '23': 61, '24'

In [24]:
Sentence.vocab.stoi['new construction'], Label.vocab.stoi['new construction'] 

(0, 0)

Load our data.

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

We'll also print out an example just to double check they're not reversed.

In [ ]:
print(vars(train.examples[0]))

{'sentence': ['Many', 'towns', 'and', 'cities', 'have', 'trash', 'cans', 'where', 'on', 'sidewalks', '?', '\\n', '(', 'A', ')', 'hospital', '(', 'B', ')', 'park', '(', 'C', ')', 'corner', '(', 'D', ')', 'motel', '(', 'E', ')', 'office'], 'label': ['corner']}


Then create our vocabulary, converting all tokens appearing less than twice into `<unk>` tokens.

In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

Finally, define the `device` and create our iterators.

In [ ]:
BATCH_SIZE = 128

train_iterator_de_en, valid_iterator_de_en, test_iterator_de_en = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
BATCH_SIZE=16
train_iterator, test_iterator = BucketIterator.splits(
    (train, valid), 
    batch_size = BATCH_SIZE, 
    device = device, sort=False, shuffle=False)

In [113]:
iter_one = next(iter(train_iterator))

In [114]:
iter_one.label.shape

torch.Size([53, 128])

In [ ]:
for entry in (iter_one.sentence[0][0,:]):
    print(entry.item())

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

## Model Checks

In [ ]:
next(iter(train_iterator_de_en)), iter_one

(
 [torchtext.data.batch.Batch of size 128 from MULTI30K]
 	[.src]:[torch.LongTensor of size 34x128]
 	[.trg]:[torch.LongTensor of size 35x128], 
 [torchtext.data.batch.Batch of size 128]
 	[.sentence]:('[torch.LongTensor of size 128x64]', '[torch.LongTensor of size 128]')
 	[.label]:('[torch.LongTensor of size 128x4]', '[torch.LongTensor of size 128]'))

In [ ]:
iter_one.sentence[1],iter_one.sentence[0][1]

NameError: ignored

In [ ]:
text, text_lengths = iter_one.sentence
embedded = nn.Embedding(len(Sentence.vocab), 256)(text)
packed_seq = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True, enforce_sorted=False) 
outputs, hidden = nn.GRU(256, 512)(packed_seq)

In [ ]:
outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
hidden, hidden_lengths = torch.nn.utils.rnn.pad_packed_sequence(hidden)

In [ ]:
hidden, outputs

In [ ]:
hidden.shape,dec_embed.shape

(torch.Size([1, 128, 512]), torch.Size([1, 128, 256]))

In [ ]:
label_text, label_text_lengths = iter_one.label
label_embedded = nn.Embedding(len(Label.vocab), 256)(label_text)
label_packed_seq = nn.utils.rnn.pack_padded_sequence(label_embedded, label_text_lengths.cpu(), batch_first=False, enforce_sorted=False) 
label_outputs, label_hidden = nn.GRU(256, 512)(label_packed_seq)

In [ ]:
embedded.shape, text.shape
non_padded_output = nn.GRU(256, 512)(embedded)


In [ ]:
print(non_padded_output[0].shape, non_padded_output[1].shape, outputs.data.shape, hidden.shape)
print(label_outputs.data.shape, label_hidden.data.shape)

torch.Size([128, 64, 512]) torch.Size([1, 64, 512]) torch.Size([4986, 512]) torch.Size([1, 128, 512])
torch.Size([207, 512]) torch.Size([1, 128, 512])


In [ ]:
len(Sentence.vocab),len(Label.vocab)

(4344, 986)

In [ ]:
iter_de_en = next(iter(train_iterator_de_en))
embedded = nn.Embedding(len(SRC.vocab), 256)(iter_de_en.src)

output_deen, hidden_deen = nn.GRU(256, 512)(embedded)

In [ ]:
hidden_deen.shape, hidden.shape

(torch.Size([1, 128, 512]), torch.Size([1, 128, 512]))

In [ ]:
label_text, label_text_lengths =iter_one.label

In [ ]:
print(label_text.shape, label_text_lengths.shape, iter_de_en.trg.shape)
label_packed_seq = nn.utils.rnn.pack_padded_sequence(label_text, label_text_lengths.cpu(), batch_first=True, enforce_sorted=False) 
label_packed_seq.sorted_indices.shape

torch.Size([128, 4]) torch.Size([128]) torch.Size([31, 128])


torch.Size([128])

In [ ]:
label_text.shape

torch.Size([128, 4])

In [ ]:
label_packed_seq.data.shape

torch.Size([207])

In [ ]:
iter_de_en.src.shape, iter_de_en.trg.shape

(torch.Size([36, 128]), torch.Size([31, 128]))

In [ ]:
label_text_reshaped =  label_text.reshape(4, -1)

In [ ]:
input_to_decoder.shape

torch.Size([128])

In [ ]:
# input_to_decoder = label_text_reshaped[0,:]
# input_to_decoder = input_to_decoder.unsqueeze(0)

# INPUT_DIM = len(Sentence.vocab)
# OUTPUT_DIM = len(Label.vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# HID_DIM = 512
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

# enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
# dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)


dec_embed = nn.Embedding(len(Label.vocab), 256)(input_to_decoder)

In [ ]:
dec_embed.shape
dec_output, dec_hidden = nn.GRU(256, 512)(dec_embed)

In [ ]:
dec_embed.squeeze(0).shape,hidden.shape

(torch.Size([128, 256]), torch.Size([1, 128, 512]))

In [ ]:
torch.cat((dec_embed, hidden), dim=2).shape

torch.Size([1, 128, 768])

In [ ]:
dec_output.shape, dec_hidden.shape, hidden.shape

(torch.Size([1, 128, 512]),
 torch.Size([1, 128, 512]),
 torch.Size([1, 128, 512]))

In [ ]:
for entry in label_packed_seq.data:
    print(Label.vocab.itos[entry.item()])

In [ ]:
iter_de_en.trg[0,:].shape, iter_one.label[0]

In [ ]:
label_text.shape

torch.Size([128, 4])

In [ ]:
label_text_reshaped[0]

tensor([196,  36,   3,   1, 393,   1,   1,   1,  35,  82,   1,   1, 971,   1,
          1,   1, 888,   1,   1,   1,  28,   1,   1,   1,  17, 300, 551,   1,
        791,  21,   1,   1, 605,   1,   1,   1, 215, 304,   1,   1,  58,  11,
          5,   1, 428,  52,   1,   1, 108,  64,   1,   1, 705,   1,   1,   1,
        549,   1,   1,   1,  51,   3,  16,   1, 148, 701,   1,   1,   6,   1,
          1,   1,  44,   1,   1,   1,  12,   1,   1,   1, 410,   1,   1,   1,
        795, 772,   1,   1, 688,   1,   1,   1, 611,   1,   1,   1, 592, 719,
          1,   1, 188,   1,   1,   1,  34, 200, 497,   1, 563, 857,   1,   1,
        601,   1,   1,   1, 771, 951,   1,   1, 665, 974,   1,   1,  25,   1,
          1,   1])

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        text, text_lengths = src
        embedded = self.dropout(self.embedding(text))
        # packed_seq = nn.utils.rnn.pack_padded_sequence(embedded, 
        #                                                text_lengths.cpu(),
        #                                                batch_first=False,
        #                                                enforce_sorted=False)        
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state! 
        ## outputs is a packed sequence but since it is not used we will not 
        ## unpack it         
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

## Model Definition

In [20]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        # print(embedded.shape, hidden.shape, context.shape)
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [21]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

## Old LSTM Code

In [ ]:
def train_lstm(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        sentence, sentence_lengths = batch.sentence
        
        # convert to 1D tensor
        predictions = model(sentence, sentence_lengths).squeeze()  
        #print(predictions)
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

# Training the Seq2Seq Model

The rest of this session is very similar to the previous one. 

We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [22]:
# INPUT_DIM = len(SRC.vocab)
# OUTPUT_DIM = len(TRG.vocab)

INPUT_DIM = len(Sentence.vocab)
OUTPUT_DIM = len(Label.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 256
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [23]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6784, 256)
    (rnn): GRU(256, 256)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(9945, 256)
    (rnn): GRU(512, 256)
    (fc_out): Linear(in_features=768, out_features=9945, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

We print out the number of parameters.

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [24]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 12,916,441 trainable parameters


We initiaize our optimizer.

In [25]:
from torch.optim.lr_scheduler import StepLR, OneCycleLR, MultiStepLR, CyclicLR, ReduceLROnPlateau

optimizer = optim.Adam(model.parameters(), lr=0.0009893272, weight_decay=0.0001)
scheduler = OneCycleLR(optimizer, 
                        0.001, 
                        epochs=100, 
                        cycle_momentum=False, 
                        steps_per_epoch=len(train_iterator), 
                        #base_momentum=config.momentum,
                        #max_momentum=0.95, 
                        pct_start=0.208,
                        # anneal_strategy=config.anneal_strategy,
                        div_factor=100,
                        # final_div_factor=config.final_div_factor
                        )
# 0.000981989011942079

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [26]:
#TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

TRG_PAD_IDX = Label.vocab.stoi[Label.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [27]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.sentence
        trg = batch.label
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        # print("output_dim:", output.shape)
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        # print("output_dim before loss:", output.shape, trg.shape)
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        # scheduler.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
iter_one.label[0].shape

torch.Size([128, 4])

In [ ]:
iter_one = next(iter(train_iterator))
iter_one.label[0][1:].view(-1)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [28]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            # src = batch.src
            # trg = batch.trg
            src = batch.sentence
            trg = batch.label
            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [32]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [30]:
optimizer = optim.Adam(model.parameters())

In [33]:
#optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.001)

In [33]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')
# model.load_state_dict(torch.load('tut2-model.pt'))
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f} | LR: {scheduler.get_last_lr()}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 39s
	Train Loss: 4.327 | Train PPL:  75.738 | LR: [1.0000000000000026e-05]
	 Val. Loss: 5.967 |  Val. PPL: 390.312
Epoch: 02 | Time: 0m 39s
	Train Loss: 3.858 | Train PPL:  47.367 | LR: [1.0000000000000026e-05]
	 Val. Loss: 5.818 |  Val. PPL: 336.325
Epoch: 03 | Time: 0m 39s
	Train Loss: 3.411 | Train PPL:  30.299 | LR: [1.0000000000000026e-05]
	 Val. Loss: 6.075 |  Val. PPL: 434.819
Epoch: 04 | Time: 0m 39s
	Train Loss: 3.004 | Train PPL:  20.160 | LR: [1.0000000000000026e-05]
	 Val. Loss: 6.234 |  Val. PPL: 509.812
Epoch: 05 | Time: 0m 39s
	Train Loss: 2.610 | Train PPL:  13.598 | LR: [1.0000000000000026e-05]
	 Val. Loss: 6.272 |  Val. PPL: 529.730
Epoch: 06 | Time: 0m 39s
	Train Loss: 2.228 | Train PPL:   9.278 | LR: [1.0000000000000026e-05]
	 Val. Loss: 6.080 |  Val. PPL: 436.870
Epoch: 07 | Time: 0m 39s
	Train Loss: 1.886 | Train PPL:   6.592 | LR: [1.0000000000000026e-05]
	 Val. Loss: 5.987 |  Val. PPL: 398.151
Epoch: 08 | Time: 0m 39s
	Train Loss: 1.560 | Tr

In [34]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [35]:
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.01)

In [36]:
best_valid_loss

5.818077982740199

In [37]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = 5.818077982740199
model.load_state_dict(torch.load('tut2-model.pt'))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 39s
	Train Loss: 3.856 | Train PPL:  47.272
	 Val. Loss: 5.698 |  Val. PPL: 298.329
Epoch: 02 | Time: 0m 39s
	Train Loss: 4.366 | Train PPL:  78.757
	 Val. Loss: 5.911 |  Val. PPL: 369.151
Epoch: 03 | Time: 0m 39s
	Train Loss: 4.646 | Train PPL: 104.201
	 Val. Loss: 5.929 |  Val. PPL: 375.843
Epoch: 04 | Time: 0m 39s
	Train Loss: 4.815 | Train PPL: 123.316
	 Val. Loss: 5.853 |  Val. PPL: 348.252
Epoch: 05 | Time: 0m 39s
	Train Loss: 4.922 | Train PPL: 137.241
	 Val. Loss: 5.793 |  Val. PPL: 327.893
Epoch: 06 | Time: 0m 39s
	Train Loss: 4.962 | Train PPL: 142.917
	 Val. Loss: 5.731 |  Val. PPL: 308.184
Epoch: 07 | Time: 0m 39s
	Train Loss: 4.991 | Train PPL: 147.156
	 Val. Loss: 5.781 |  Val. PPL: 324.109
Epoch: 08 | Time: 0m 39s
	Train Loss: 5.010 | Train PPL: 149.879
	 Val. Loss: 5.730 |  Val. PPL: 308.008
Epoch: 09 | Time: 0m 39s
	Train Loss: 5.024 | Train PPL: 152.050
	 Val. Loss: 5.707 |  Val. PPL: 300.955
Epoch: 10 | Time: 0m 39s
	Train Loss: 5.030 | Train PPL

In [39]:
print(best_valid_loss)

5.566523430195261


In [43]:
optimizer = optim.Adam(model.parameters(), lr=0.000001, weight_decay=0.01)

In [44]:
N_EPOCHS = 20
CLIP = 1
print(best_valid_loss)
model.load_state_dict(torch.load('tut2-model.pt'))

best_valid_loss = 5.566523430195261 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

5.566523430195261
Epoch: 01 | Time: 0m 14s
	Train Loss: 5.269 | Train PPL: 194.203
	 Val. Loss: 5.566 |  Val. PPL: 261.344
Epoch: 02 | Time: 0m 14s
	Train Loss: 5.274 | Train PPL: 195.217
	 Val. Loss: 5.566 |  Val. PPL: 261.502
Epoch: 03 | Time: 0m 14s
	Train Loss: 5.274 | Train PPL: 195.126
	 Val. Loss: 5.566 |  Val. PPL: 261.268
Epoch: 04 | Time: 0m 14s
	Train Loss: 5.273 | Train PPL: 195.009
	 Val. Loss: 5.568 |  Val. PPL: 261.844
Epoch: 05 | Time: 0m 14s
	Train Loss: 5.272 | Train PPL: 194.738
	 Val. Loss: 5.568 |  Val. PPL: 261.865
Epoch: 06 | Time: 0m 14s
	Train Loss: 5.273 | Train PPL: 194.975
	 Val. Loss: 5.568 |  Val. PPL: 261.885
Epoch: 07 | Time: 0m 14s
	Train Loss: 5.273 | Train PPL: 194.943
	 Val. Loss: 5.568 |  Val. PPL: 261.903
Epoch: 08 | Time: 0m 14s
	Train Loss: 5.270 | Train PPL: 194.457
	 Val. Loss: 5.568 |  Val. PPL: 261.942
Epoch: 09 | Time: 0m 14s
	Train Loss: 5.277 | Train PPL: 195.818
	 Val. Loss: 5.568 |  Val. PPL: 261.943
Epoch: 10 | Time: 0m 14s
	Train Loss:

In [47]:
best_valid_loss

5.654881441846807

#### Final Perplexity = 
Epoch: 16 | Time: 0m 14s
	Train Loss: 5.290 | Train PPL: 198.310
	 Val. Loss: 5.568 |  Val. PPL: 261.804

In [48]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.655 | Test PPL: 285.683 |


Just looking at the test loss, we get better performance. This is a pretty good sign that this model architecture is doing something right! Relieving the information compression seems like the way forard, and in the next tutorial we'll expand on this even further with *attention*.